In [1]:
## anaconda3 (Python 3.12.0) Kernel
import numpy as np

# pair trade packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from datetime import datetime

# Load Pairs Data


In [2]:
def custom_date_parser(date_str):
    return datetime.strptime(date_str, '%d/%m/%Y')

# Load the dictionary from the pickle file
with open('pairsOutcome.pkl', 'rb') as file:
    pairsOutcome = pickle.load(file)

print("Dictionary loaded from pairsOutcome.pkl")


# Load stock data and get return 
tpxData = pd.read_csv('TPX_prices.csv', index_col=0, parse_dates=True, date_parser=custom_date_parser)
tpxData = tpxData.dropna(axis='columns')
return_df = (tpxData / tpxData.shift(1)) - 1

Dictionary loaded from pairsOutcome.pkl


/var/folders/h2/r7qn2m9n1zb6y_0q191gdqth0000gn/T/ipykernel_95892/3199335484.py:12: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  tpxData = pd.read_csv('TPX_prices.csv', index_col=0, parse_dates=True, date_parser=custom_date_parser)


# Get Pair Trade Portfolio
`pairsOutcome` already have TOPIX stocks with highest liquidity and are tested for stationarity over a 1 year window

Choose top 10 known pair trades by returns in the total dataset

In [3]:
# Sort the keys by their cumpnl[-2] values in descending order
top_keys = sorted(
    pairsOutcome,
    key=lambda k: pairsOutcome[k].cumpnl.iloc[-2],  # Access cumpnl[-2] safely
    reverse=True
)[:10]  # Get the top 10 keys

# Print the top 10 performing trades
print("Top 10 performing trades:")
for i, key in enumerate(top_keys, 1):
    print(f"{i}. Key: {key}, Value: {pairsOutcome[key].cumpnl.iloc[-2]}")

Top 10 performing trades:
1. Key: 1801 JP Equity 2670 JP Equity, Value: 2.5797887367591246
2. Key: 3778 JP Equity 6701 JP Equity, Value: 2.537242032391529
3. Key: 2760 JP Equity 6254 JP Equity, Value: 2.3688208386917404
4. Key: 5706 JP Equity 6954 JP Equity, Value: 2.2676474298290237
5. Key: 7951 JP Equity 9684 JP Equity, Value: 2.0657325467200596
6. Key: 1808 JP Equity 6481 JP Equity, Value: 1.9929348941248262
7. Key: 3099 JP Equity 5831 JP Equity, Value: 1.939742664925484
8. Key: 1808 JP Equity 6971 JP Equity, Value: 1.9132602773493155
9. Key: 4021 JP Equity 9843 JP Equity, Value: 1.8675031161000868
10. Key: 5929 JP Equity 6504 JP Equity, Value: 1.811533049967201


In [4]:
## Get pair stock data
def custom_date_parser(date_str):
    return datetime.strptime(date_str, '%d/%m/%Y')
valid = pd.read_csv('validPairs4.csv', 
                    index_col=0, 
                    parse_dates=True, 
                    date_parser=custom_date_parser)
## get list of pair stocks
validPairsList = [
    [item.strip() + ' Equity' for item in pair.split('Equity') if item.strip()]
    for pair in top_keys
]

/var/folders/h2/r7qn2m9n1zb6y_0q191gdqth0000gn/T/ipykernel_95892/199844939.py:4: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  valid = pd.read_csv('validPairs4.csv',


In [5]:
rollingWindow = 262
cutLossSd = 2

In [6]:
for pair in validPairsList:
    df = pd.DataFrame()

    #Calculate Standard Deviations
    df['spread'] = valid[f'spread_{pair[0]}_{pair[1]}']
    df['mid'] =  df['spread'].rolling(rollingWindow).mean()
    df['1sd high'] = df['spread'].rolling(rollingWindow).mean() + df['spread'].rolling(rollingWindow).std()
    df['1sd low'] = df['spread'].rolling(rollingWindow).mean() - df['spread'].rolling(rollingWindow).std()
    df['2sd high'] = df['spread'].rolling(rollingWindow).mean() + df['spread'].rolling(rollingWindow).std() * cutLossSd
    df['2sd low'] = df['spread'].rolling(rollingWindow).mean() - df['spread'].rolling(rollingWindow).std() * cutLossSd
    df['position'] = 0

    df.loc[(df['spread'] > df['1sd high']) & (df['spread'] < df['2sd high']), 'position'] = -1
    df.loc[(df['spread']< df['1sd low']) & (df['spread'] > df['2sd low']), 'position'] = 1

    #Calculate PnL
    df[f'{pair[0]} position'] = df['position']
    df[f'{pair[1]} position'] = df['position'] * -1
    df['dailypnl'] = df[f'{pair[1]} position']*return_df[f'{pair[1]}'].shift(-1) + df[f'{pair[0]} position']*return_df[f'{pair[0]}'].shift(-1)
    df['cumpnl'] = df['dailypnl'].cumsum()

    pairsOutcome[f'{pair[0]} {pair[1]}'] = df

## Make indicators and spread stationary around 0
Deduct the mean from all values to translate to 0 axis

In [7]:
workingPairOutcome = {}

for pair in top_keys:
    dummy_df = pairsOutcome[top_keys[0]].iloc[::,:6]
    dummy_df = dummy_df.subtract(dummy_df['mid'], axis=0).drop(columns=['mid']) # centre spread and SD
    dummy_df = dummy_df.div(dummy_df['2sd high']-dummy_df['1sd high'],axis=0)   # express SD as integers, give spread as propotionate
    dummy_df['1sd_high_boolean'] = (dummy_df['spread']>dummy_df['1sd high']).astype(int)
    dummy_df['2sd_high_boolean'] = (dummy_df['spread']>dummy_df['2sd high']).astype(int)
    dummy_df['1sd_low_boolean'] =  (dummy_df['spread']<dummy_df['1sd low'] ).astype(int)
    dummy_df['2sd_low_boolean'] =  (dummy_df['spread']<dummy_df['2sd low'] ).astype(int)
    dummy_df = dummy_df.drop(columns=['spread','1sd high', '1sd low', '2sd high', '2sd low'])
    workingPairOutcome[pair] = dummy_df.to_numpy()

In [8]:
workingPairOutcome[top_keys[5]][-5:]     # spread is not a proportion and direction of SD

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]])

# Machine Learning Challenge

## Background
Initial evaluation of the baseline portfolio shows that draw downs are small. Originally team had the idea of using Machine Learning to optimise for sizing of these pair trades. However since there was no significant drawdowns the returns are linearly increasing with investment sizing i.e. greater nominal investment in the the pair trade the proportionate increase in returns without realising significant drawdown risk.

Instead of optimising for sizing, we can explore Machine Learning in terms of strategy on this stationary dataset. Whereas our prescribed strategy is to enter at +/- 1 std dev, exit at 0 with +/- 2 std dev stop loss. These are only suggestions and arbitrary levels.

With Machine Learning, we can discover if it will uncover the mean reverting nature and recommend another threshhold. We use Q Learner to understand state space with the same spread, mid, std dev parameters as the baseline.

### Q Value table

In [9]:
workingPairOutcome[top_keys[0]][261]

array([1, 0, 0, 0])

In [15]:
df = pairsOutcome[top_keys[0]]
df.tail()

,spread,mid,1sd high,1sd low,2sd high,2sd low,position,1801 JP Equity position,2670 JP Equity position,dailypnl,cumpnl
Date,,,,,,,,,,,
2024-05-27,-39.304773,-46.289714,319.903354,-412.482783,686.096423,-778.675852,0,0,0,0.00000,2.552379
2024-05-28,204.731719,-44.231112,321.850855,-410.313080,687.932823,-776.395048,0,0,0,-0.00000,2.552379
2024-05-29,345.042254,-41.541554,324.802844,-407.885953,691.147243,-774.230351,-1,-1,1,0.02741,2.579789
2024-05-30,178.588811,-39.334254,326.585734,-405.254242,692.505722,-771.174230,0,0,0,0.00000,2.579789
2024-05-31,15.040724,-37.437415,327.466148,-402.340979,692.369711,-767.244542,0,0,0,NaN,NaN


In [16]:
ls_pos = [-1,0,1]
pair = validPairsList[0]

#Calculate PnL
for pos in ls_pos:
    df[f'dailypnl_{pos}'] = pos*return_df[f'{pair[0]}'].shift(-1) + -pos*return_df[f'{pair[1]}'].shift(-1)

In [17]:
df

,spread,mid,1sd high,1sd low,2sd high,2sd low,position,1801 JP Equity position,2670 JP Equity position,dailypnl,cumpnl,dailypnl_-1,dailypnl_0,dailypnl_1
Date,,,,,,,,,,,,,,
2013-01-01,-829.459706,NaN,NaN,NaN,NaN,NaN,0,0,0,0.00000,0.000000,0.000000,0.0,0.000000
2013-01-02,-829.459706,NaN,NaN,NaN,NaN,NaN,0,0,0,0.00000,0.000000,0.000000,0.0,0.000000
2013-01-03,-829.459706,NaN,NaN,NaN,NaN,NaN,0,0,0,0.00000,0.000000,-0.030114,0.0,0.030114
2013-01-04,-788.012196,NaN,NaN,NaN,NaN,NaN,0,0,0,0.00000,0.000000,-0.015326,0.0,0.015326
2013-01-07,-751.666698,NaN,NaN,NaN,NaN,NaN,0,0,0,-0.00000,0.000000,0.012865,-0.0,-0.012865
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-27,-39.304773,-46.289714,319.903354,-412.482783,686.096423,-778.675852,0,0,0,0.00000,2.552379,-0.040053,0.0,0.040053
2024-05-28,204.731719,-44.231112,321.850855,-410.313080,687.932823,-776.395048,0,0,0,-0.00000,2.552379,-0.022316,-0.0,0.022316
2024-05-29,345.042254,-41.541554,324.802844,-407.885953,691.147243,-774.230351,-1,-1,1,0.02741,2.579789,0.027410,0.0,-0.027410


In [22]:
# within 1 to 2 high SD
df.loc[(df['spread'] > df['1sd high']) & (df['spread'] < df['2sd high'])][['dailypnl_-1', 'dailypnl_0', 'dailypnl_1']].sum()

dailypnl_-1    1.491959
dailypnl_0     0.000000
dailypnl_1    -1.491959
dtype: float64

In [24]:
# within 1 to 2 high SD
df.loc[(df['spread'] < df['1sd low']) & (df['spread'] > df['2sd low'])][['dailypnl_-1', 'dailypnl_0', 'dailypnl_1']].sum()

dailypnl_-1   -1.08783
dailypnl_0     0.00000
dailypnl_1     1.08783
dtype: float64